In [3]:
import heapq  # For priority queue in A* algorithm
import random  # To place obstacles and the trapped character randomly
import math    # For distance calculation in the heuristic

# Function to create the grid and randomly place the trapped character
def create_grid(size):
    grid = [[' ' for _ in range(size)] for _ in range(size)]  # Initialize grid with empty spaces
    grid[0][0] = 'S'  # Start point

    # Place trapped character ('R') at a random location
    trapped_x, trapped_y = random.randint(0, size-1), random.randint(0, size-1)
    while (trapped_x, trapped_y) == (0, 0):  # Ensure it's not at the start
        trapped_x, trapped_y = random.randint(0, size-1), random.randint(0, size-1)

    grid[trapped_x][trapped_y] = 'R'  # Mark the trapped character
    return grid, (trapped_x, trapped_y)  # Return grid and goal position

# Add random obstacles to the grid
def add_obstacles(grid, num_obstacles):
    size = len(grid)
    for _ in range(num_obstacles):
        obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        while grid[obstacle_x][obstacle_y] in ['S', 'R']:  # Avoid placing obstacles on start and goal
            obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[obstacle_x][obstacle_y] = 'X'  # Mark obstacle
    return grid

# Check if a position is valid for movement
def is_valid_position(grid, x, y):
    size = len(grid)
    return 0 <= x < size and 0 <= y < size and grid[x][y] != 'X'  # Ensure the position is within bounds and not blocked

# Heuristic function: Straight-line distance (Euclidean distance)
def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

# A* algorithm implementation
def a_star(grid, start, goal):
    size = len(grid)
    open_list = []
    heapq.heappush(open_list, (0, start))  # Push the start position to the priority queue

    g_score = {start: 0}  # Initialize g_score (cost to reach each node)
    parent = {start: None}  # For path reconstruction

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Possible directions (up, down, left, right)

    while open_list:
        _, current = heapq.heappop(open_list)  # Get the node with the lowest cost

        if current == goal:
            break  # Stop if goal is reached

        for direction in directions:
            next_x = current[0] + direction[0]
            next_y = current[1] + direction[1]
            next_state = (next_x, next_y)

            if is_valid_position(grid, next_x, next_y):
                tentative_g_score = g_score[current] + 1  # Assume uniform cost for moving

                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    g_score[next_state] = tentative_g_score
                    f_score = tentative_g_score + heuristic(next_state, goal)
                    heapq.heappush(open_list, (f_score, next_state))
                    parent[next_state] = current  # Set parent for path reconstruction

    # Reconstruct the path
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = parent[current]
    path.reverse()  # Return the path from start to goal
    return path

# Print the grid with the path marked
def print_grid_with_path(grid, path):
    grid_with_path = [row.copy() for row in grid]  # Copy the grid to keep the original unchanged
    for (x, y) in path:
        if grid_with_path[x][y] not in ['S', 'R']:  # Don't overwrite start and goal
            grid_with_path[x][y] = '*'  # Mark the path

    # Print the grid
    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))  # Print top border line
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')  # Print each row with vertical borders
        print('-' * (len(grid_with_path) * 4 + 1))  # Print horizontal line after each row

# Main function to run the game
def rescue_mission():
    # Get grid size and number of obstacles from the user
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))
    num_obstacles = int(input(f"Enter the number of obstacles (less than {size * size - 2}): "))

    # Create grid and place the trapped character
    grid, goal = create_grid(size)
    grid = add_obstacles(grid, num_obstacles)  # Add obstacles

    # Print the initial grid
    print("\nInitial Grid:")
    print_grid_with_path(grid, [])  # No path yet

    # Use A* algorithm to find the shortest path
    print("\nSearching for the trapped character using A* algorithm...\n")
    path = a_star(grid, (0, 0), goal)  # Start from top-left corner

    # Print the final grid with the path
    print("\nPath to the Trapped Character:")
    print_grid_with_path(grid, path)

# Run the game
rescue_mission()


Enter the grid size (e.g., 6 for a 6x6 grid): 3
Enter the number of obstacles (less than 7): 3

Initial Grid:

Grid with Path:
-------------
| S | R |   |
-------------
| X |   |   |
-------------
|   |   | X |
-------------

Searching for the trapped character using A* algorithm...


Path to the Trapped Character:

Grid with Path:
-------------
| S | R |   |
-------------
| X |   |   |
-------------
|   |   | X |
-------------
